In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize

In [3]:
img_size= 256
batch_size= 32
channels= 3
epochs= 6

Constants have been set successfully!!

In [4]:
dir_path="potato2"
class_names= os.listdir(dir_path)
class_names

['Potato___Early_Blight', 'Potato___Healthy', 'Potato___Late_Blight']

In [5]:
le= LabelEncoder() #holds the label for each class
le.fit(class_names) #class names have been fit into an array
le.classes_

array(['Potato___Early_Blight', 'Potato___Healthy',
       'Potato___Late_Blight'], dtype='<U21')

In [6]:
x,y=[],[]

for c in class_names:
    class_dir= os.path.join(dir_path,c)
    for f in os.listdir(class_dir):
        img= Image.open(os.path.join(class_dir,f))
        img= img.resize((img_size,img_size))
        img= np.array(img)/ 255.0 #normalize pixel value (8-bit image)
        x.append(img)
        y.append(c)

In [7]:
x= np.array(x)
y= le.transform(y) #transforms the class names into labels

Split the dataset into Training, Validation and Test Sets

In [8]:
x_train,x_temp,y_train,y_temp= train_test_split(x, y, test_size=0.2, random_state=123)
x_val,x_test,y_val,y_test= train_test_split(x_temp,y_temp, test_size=0.5, random_state=123)
#80% data for Training
#10% data for validation
#10% data for ttesting

Data Visualization

In [ ]:
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_train[i])
    plt.title(class_names[y_train[i]])
    plt.axis("off")
    
plt.show

<function matplotlib.pyplot.show(close=None, block=None)>

Shuffle and Cache the Datasets

In [81]:
#Shuffle the training dataset
shuffle_ind= np.arange(len(x_train))
np.random.shuffle(shuffle_ind)
x_train= x_train[shuffle_ind]
y_train= y_train[shuffle_ind]

In [82]:
#Shuffle the validation dataset
shuffle_ind= np.arange(len(x_val))
np.random.shuffle(shuffle_ind)
x_val= x_val[shuffle_ind]
y_val= y_val[shuffle_ind]

In [83]:
#Shuffle the test dataset
shuffle_ind= np.arange(len(x_test))
np.random.shuffle(shuffle_ind)
x_test= x_test[shuffle_ind]
y_test= y_test[shuffle_ind]

In [84]:
#creating data generators to iterate over the data in batches (Prefetching)
def data_generator(x,y,batch_size):
    samples=len(x)
    while True:
        for i in range(0,samples,batch_size):
            x_batch= x[i:i + batch_size]
            y_batch= y[i:i + batch_size]
            yield x_batch,y_batch

            
#iterating over the data using generators
train_dg= data_generator(x_train,y_train,batch_size)
val_dg= data_generator(x_val,y_val,batch_size)
test_dg= data_generator(x_test,y_test,batch_size)

Model Architecture (CNN) 

In [85]:
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / e_x.sum(axis=-1, keepdims=True)


In [95]:
class convLayer:
    def __init__(self, filters, kernels, channels, activation):
        self.filters = filters
        self.kernels = kernels
        self.activation = activation
        self.weights = np.random.randn(kernels, kernels, channels, filters) * 0.01
        self.bias = np.zeros((filters,))
        
    def forward(self, inputs):
        batch_size, input_height, input_width, input_channels = inputs.shape
        output_height = input_height - self.kernels + 1
        output_width = input_width - self.kernels + 1
        
        self.output = np.zeros((batch_size, output_height, output_width, self.filters))
        
        for i in range(batch_size):
            for a in range(output_height):
                for b in range(output_width):
                    for j in range(self.filters):
                        # Extract the corresponding input slice
                        input_slice = inputs[i, a:a + self.kernels, b:b + self.kernels, :]
                        # Perform element-wise multiplication and sum
                        self.output[i, a, b, j] = np.sum(input_slice * self.weights[:, :, :, j]) + self.bias[j]
            
        self.output = self.activation(self.output)
        return self.output


In [96]:
class maxpoolLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size
        
    def forward(self, inputs):
        batch_size, input_height, input_width, input_channels = inputs.shape
        output_height = input_height // self.pool_size
        output_width = input_width // self.pool_size
        
        self.output = np.zeros((batch_size, output_height, output_width, input_channels))
        
        for i in range(batch_size):
            for a in range(0, input_height, self.pool_size):
                for b in range(0, input_width, self.pool_size):
                    for c in range(input_channels):
                        self.output[i, a // self.pool_size, b // self.pool_size, c] = np.max(inputs[i, a:a + self.pool_size, b:b + self.pool_size, c])
        
        return self.output

In [97]:
class flattenLayer:
    def forward(self,inputs):
        self.inputs_shape=inputs.shape
        return inputs.reshape(inputs.shape[0],-1)

In [98]:
class denseLayer:
    def __init__(self,num_units,activation):
        self.num_units = num_units
        self.activation = activation
        self.weights = np.random.randn(num_units) * 0.01
        self.bias = np.zeros((num_units,))
        
    def forward(self,inputs):
        self.inputs= inputs
        self.output= np.dot(inputs,self.weights)+self.bias
        self.output= self.activation(self.output)
        return self.output

Creating the CNN Model

In [99]:
model=[
    convLayer(32,3,channels,relu),
    maxpoolLayer(2),
    convLayer(64,3,channels,relu),
    maxpoolLayer(2),
    convLayer(64,3,channels,relu),
    maxpoolLayer(2),
    convLayer(64,3,channels,relu),
    maxpoolLayer(2),
    convLayer(64,3,channels,relu),
    maxpoolLayer(2),
    flattenLayer(),
    denseLayer(64,relu),
    denseLayer(len(class_names),softmax)
]

Loss Function

In [100]:
def categorical_crossentropy(predictions,targets):
    epsilon= 1e-15
    predictions= np.clip(predictions, epsilon, 1-epsilon)
    n= predictions.shape[0]
    loss= -np.sum(targets* np.log(predictions+1e-9))/n
    return loss

Gradient Descent Function

In [101]:
def gradient_descent(params,gradients,learning_rate=0.001):
    for p,g in zip(params,gradients):
        p-=learning_rate*g


Training Parameters

In [102]:
learning_rate=0.001

Training Loop

In [ ]:
def convolution(input_data, filters, stride=1, padding='valid'):
    if padding == 'valid':
        output_height = (input_data.shape[0] - filters.shape[0]) // stride + 1
        output_width = (input_data.shape[1] - filters.shape[1]) // stride + 1
    elif padding == 'same':
        output_height = input_data.shape[0]
        output_width = input_data.shape[1]
    else:
        raise ValueError("Invalid padding mode. Use 'valid' or 'same'.")

    output = np.zeros((output_height, output_width))

    for i in range(0, input_data.shape[0] - filters.shape[0] + 1, stride):
        for j in range(0, input_data.shape[1] - filters.shape[1] + 1, stride):
            output[i, j] = np.sum(input_data[i:i+filters.shape[0], j:j+filters.shape[1]] * filters)

    return output


# Define a function for a custom convolutional layer
def custom_conv_layer(input_data, filters, bias, stride=1, padding='valid'):
    # Perform convolution operation and apply activation (e.g., ReLU)
    conv_result = convolution(input_data, filters, stride, padding)
    activation_result = relu(conv_result)  # Apply ReLU activation
    return activation_result

# Define a function for a custom dense (fully connected) layer
def custom_dense_layer(input_data, weights, bias):
    # Perform matrix multiplication and apply activation (e.g., ReLU)
    dense_result = np.dot(input_data, weights) + bias
    activation_result = relu(dense_result)  # Apply ReLU activation
    return activation_result

# Define a function to compute gradients for a layer
def compute_layer_gradients(layer, prev_activation, delta):
    # Compute gradients for weights and bias
    if hasattr(layer, 'weights') and hasattr(layer, 'bias'):
        gradients = {
            'weights': np.dot(prev_activation.T, delta),
            'bias': np.sum(delta, axis=0)
        }
    else:
        gradients = None
    return gradients



for epoch in range(epochs):
    total_loss = 0
    num_batches = 0  # Initialize a counter for the number of batches processed in this epoch
    
    for x_batch, y_batch in train_dg:
        # Forward pass through your model
        output = x_batch
        activations = [output]  # List to store layer activations
        
        for layer in model:
            output = layer.forward(output)
            activations.append(output)
        
        # Calculate the categorical cross-entropy loss
        loss = categorical_crossentropy(output, y_batch)
        total_loss += loss
        num_batches += 1  # Increment the batch counter
        
        # Compute gradients using backpropagation
        delta = output - y_batch  # Gradient of loss with respect to the output of the last layer
        all_gradients = []
        
        for i in reversed(range(len(model))):
            layer = model[i]
            prev_activation = activations[i]
            
            # Compute gradients for the current layer
            layer_gradients = compute_layer_gradients(layer, prev_activation, delta)
            all_gradients.append(layer_gradients)
            
            # Compute gradient of loss with respect to the input of the current layer
            delta = layer.backward(delta)
        
        # Reverse the gradients list to match the layer order
        all_gradients.reverse()
        
        # Update model parameters
        for layer, layer_gradients in zip(model, all_gradients):
            if layer_gradients:
                layer.weights -= learning_rate * layer_gradients['weights']
                layer.bias -= learning_rate * layer_gradients['bias']

    # Calculate and print average loss for this epoch
    average_loss = total_loss / num_batches  # Use num_batches to calculate average loss
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}")